In [121]:
import pandas as pd
import torch
import numpy as np
from datetime import datetime

In [129]:
data = pd.read_csv("../Current_Data/New_data_updated_names.csv", usecols=['Country','Date','Cases','Deaths','Summary'], 
                  parse_dates=["Date"])
data.head()

,Country,Date,Cases,Deaths,Summary
0,Benin,2013-12-30,11,0.0,Two districts crossed the alert threshold at t...
1,Burkina Faso,2013-12-30,88,13.0,Two districts crossed the alert threshold at t...
2,Côte d'Ivoire,2013-12-30,3,2.0,Two districts crossed the alert threshold at t...
3,Ghana,2013-12-30,3,0.0,Two districts crossed the alert threshold at t...
4,Mali,2013-12-30,0,0.0,Two districts crossed the alert threshold at t...


In [130]:
country_list = ['Benin', 'Burkina Faso', 'Burundi', 'Cameroun', 'Centrafrique', "Ivory Coast", 'Ethiopia', 'Ghana',
               'Guinea', 'Guinea-Bissau', 'Gambia', 'Kenya', 'Mali', 'Mauritania', 'Niger', 'Nigeria', 'Dem. Rep. Congo', 
                'Senegal' , 'South Sudan', 'Sudan', 'Sierra Lone', 'Tanzania', 'Chad', 'Togo', 'Uganda']
bordering_countries = {"Benin": ["Togo", "Nigeria", "Niger", "Burkina Faso"], 
                       "Burkina Faso": ["Mali", "Niger", "Benin" ,"Togo", "Ghana", "Ivory Coast"], 
                       "Burundi": ["Dem. Rep. Congo", "Tanzania"], 
                       "Cameroun": ["Centrafrique", "Nigeria", "Guinea", "Chad"], 
                       "Centrafrique": ["Cameroun", "Sudan", "South Sudan","Dem. Rep. Congo", "Chad"], 
                       "Ivory Coast": ["Ghana", "Mali", "Burkina Faso"], 
                       "Ethiopia": ["Sudan", "South Sudan", "Kenya"], 
                       "Ghana":["Guinea", "Ivory Coast", "Burkina Faso", "Togo"], 
                       "Guinea":["Ivory Coast", "Guinea Bissau", "Mali", "Senegal", "Sierra Lone"], 
                       "Guinea Bissau": ["Senegal", "Guinea"], 
                       "Gambia":["Senegal"], 
                       "Kenya": ["Ethiopia", "South Sudan", "Uganda"],
                       "Mali": ["Niger", "Burkina Faso", "Ivory Coast", "Guinea", "Mauritania", "Senegal"], 
                       "Mauritania":["Mali", "Senegal"], 
                       "Niger": ["Benin", "Nigeria", "Burkina Faso", "Mali", "Chad"], 
                       "Nigeria": ["Benin", "Cameroun", "Niger", "Chad"], 
                       "Dem. Rep. Congo": ["Burundi", "Centrafrique", "South Sudan", "Tanzania", "Uganda"], 
                       "Senegal":["Gambia", "Guinea", "Guinea Bissau", "Mali","Mauritania"], 
                       "South Sudan":["Sudan", "Ethiopia", "Kenya", "Dem. Rep. Congo", "Uganda","Centrafrique"], 
                       "Sudan":["Ethiopia", "South Sudan", "Centrafrique", "Chad"], 
                       "Sierra Lone": ["Guinea"], 
                       "Tanzania":["Kenya", "Uganda", "Burundi", "Dem. Rep. Congo"], 
                       "Chad":["Sudan", "Centrafrique", "Cameroun", "Niger", "Nigeria"], 
                       "Togo":["Ghana", "Burkina Faso","Benin", "Guinea"],
                       "Uganda":["Kenya", "South Sudan", "Dem. Rep. Congo", "Tanzania"]}
#this lambda function returns the index of the country from the corresponding country_list array
get_indices = lambda x: country_list.index(str(x).rstrip())

In [131]:
#creates a list of bordering countries for each datapoint with the countries represented as indices of country_list array
data_bordering_countries = pd.DataFrame(data["Country"].apply(lambda x: 
                                                              list(
                                                                  map(get_indices, 
                                                                      bordering_countries[str(x).rstrip()]))
                                                                   ))
data.insert(5, "Bordering Countries", data_bordering_countries)
#replace country names in country column with corresponding indices from country_list array
data["Country"] = data["Country"].apply(lambda x: country_list.index(str(x).rstrip()))
data.head()

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2013-12-30,11,0.0,Two districts crossed the alert threshold at t...,"[23, 15, 14, 1]"
1,1,2013-12-30,88,13.0,Two districts crossed the alert threshold at t...,"[12, 14, 0, 23, 7, 5]"
2,5,2013-12-30,3,2.0,Two districts crossed the alert threshold at t...,"[7, 12, 1]"
3,7,2013-12-30,3,0.0,Two districts crossed the alert threshold at t...,"[8, 5, 1, 23]"
4,12,2013-12-30,0,0.0,Two districts crossed the alert threshold at t...,"[14, 1, 5, 8, 13, 17]"


In [139]:
def find_next_batch(count):
    initialCount = count
    found = False
    while not found:
        count += 1
        if data["Date"].loc[count + 1] != data["Date"].loc[count]:
            found = True
            break
    return data.iloc[initialCount:count+1]

In [140]:
find_next_batch(0)

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2013-12-30,11,0.0,Two districts crossed the alert threshold at t...,"[23, 15, 14, 1]"
1,1,2013-12-30,88,13.0,Two districts crossed the alert threshold at t...,"[12, 14, 0, 23, 7, 5]"
2,5,2013-12-30,3,2.0,Two districts crossed the alert threshold at t...,"[7, 12, 1]"
3,7,2013-12-30,3,0.0,Two districts crossed the alert threshold at t...,"[8, 5, 1, 23]"
4,12,2013-12-30,0,0.0,Two districts crossed the alert threshold at t...,"[14, 1, 5, 8, 13, 17]"
5,13,2013-12-30,0,0.0,Two districts crossed the alert threshold at t...,"[12, 17]"
6,15,2013-12-30,0,0.0,Two districts crossed the alert threshold at t...,"[0, 3, 14, 22]"
7,17,2013-12-30,3,0.0,Two districts crossed the alert threshold at t...,"[10, 8, 9, 12, 13]"
8,22,2013-12-30,7,0.0,Two districts crossed the alert threshold at t...,"[19, 4, 3, 14, 15]"
9,23,2013-12-30,7,0.0,Two districts crossed the alert threshold at t...,"[7, 1, 0, 8]"


In [142]:
find_next_batch(10)

,Country,Date,Cases,Deaths,Summary,Bordering Countries
10,0,2014-01-06,14,3.0,At the second week of the 2014 epidemic season...,"[23, 15, 14, 1]"
11,1,2014-01-06,97,9.0,At the second week of the 2014 epidemic season...,"[12, 14, 0, 23, 7, 5]"
12,3,2014-01-06,10,2.0,At the second week of the 2014 epidemic season...,"[4, 15, 8, 22]"
13,7,2014-01-06,16,1.0,At the second week of the 2014 epidemic season...,"[8, 5, 1, 23]"
14,8,2014-01-06,4,1.0,At the second week of the 2014 epidemic season...,"[5, 9, 12, 17, 20]"
15,12,2014-01-06,1,0.0,At the second week of the 2014 epidemic season...,"[14, 1, 5, 8, 13, 17]"
16,13,2014-01-06,0,0.0,At the second week of the 2014 epidemic season...,"[12, 17]"
17,15,2014-01-06,3,0.0,At the second week of the 2014 epidemic season...,"[0, 3, 14, 22]"
18,17,2014-01-06,7,0.0,At the second week of the 2014 epidemic season...,"[10, 8, 9, 12, 13]"
19,19,2014-01-06,0,0.0,At the second week of the 2014 epidemic season...,"[6, 18, 4, 22]"


In [144]:
data.loc[]["Country"]

0